In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
pd.set_option('display.width', 100)
pd.set_option('display.max_columns', None)

import seaborn as sea
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "browser"

from nanoporePipelineCommon import *

print("imports done")

imports done


In [2]:
reads_df_genes_raw, compressed_df_genes_raw = load_and_merge_lib_parquets(["xrn-1-5tera", "xrn-1-5tera-smg-6"], drop_sub_n=1, add_tail_groupings=False, drop_failed_polya=False, group_by_t5=True)
print("done.")

Loading readAssignments file from: /data16/marcus/genomes/elegansRelease100/Caenorhabditis_elegans.WBcel235.100.allChrs.parquet... Done.
Looking for files for libraries: ['xrn-1-5tera', 'xrn-1-5tera-smg-6']
Looking for file for xrn-1-5tera, at /data16/marcus/working/211118_nanoporeRun_totalRNA_5108_xrn-1-KD_5TERA/output_dir/merge_files/*_mergedOnReads.parquet... File Found.
Looking for file for xrn-1-5tera-smg-6, at /data16/marcus/working/211210_nanoporeRun_totalRNA_2102_xrn-1-KD_5TERA/output_dir/merge_files/*_mergedOnReads.parquet... File Found.
Loading parquet for xrn-1-5tera lib... Done.
'original_chr_pos' column already found in dataframe, skipping adjustment for 5'ends!
Loading parquet for xrn-1-5tera-smg-6 lib... Done.
'original_chr_pos' column already found in dataframe, skipping adjustment for 5'ends!
Finished assignment merge!. . .
Read counts post gene assignment:  937835
Read counts post unassigned drop:  636409
Creating groupby dataframe merged on: ['lib', 'chr_id', 'gene_i

Counting reads per gene: 100%|██████████| 31278/31278 [00:00<00:00, 66315.60it/s]


Gene counts pre sub-1 gene_hits drop:  31278
Gene counts post sub-1 gene_hits drop:  31278
done.


In [3]:
reads_df = reads_df_genes_raw.copy()
compressed_df = compressed_df_genes_raw.copy()

In [4]:
reads_df.query("gene_name == 'ubl-1'")

,lib,read_id,chr_id,chr_pos,gene_id,gene_name,cigar,sequence,polya_length,strand,t5,read_length
168006,xrn-1-5tera,9174ad35-5750-4eb6-8818-d0743f25db54,III,3068306,WBGene00006725,ubl-1,4S8M1D14M2I14M1D11M2D23M1I46M1D8M1D8M1I19M1I5M...,CCAAGTTTTTTGAGAACTAGGGTCAGTCTCAAGCATGGTCTTATCA...,50.14,+,-,578
168007,xrn-1-5tera,46a0b212-ba6f-4221-a8fc-5e0c1ef94bdc,III,3068306,WBGene00006725,ubl-1,4S8M1D6M2D6M2I24M1D20M1D18M1D12M1D20M2D5M2D7M3...,CCAAGTTTTATGAGAACTTGTCAGTCTCAAGCATGGTCTTCGTTAA...,NaN,+,-,676
168008,xrn-1-5tera,3f336577-1adc-421f-a13d-49934aa5e275,III,3068308,WBGene00006725,ubl-1,5S6M1D7M1D18M1D10M1I5M1I61M1D13M2I16M1D20M1D16...,CCAAGTTTTTGAGAACTAGGTCTGATAAGCATGGTCTCGTTAAGAC...,45.16,+,-,547
168009,xrn-1-5tera,86acf6bb-5f9b-4f7a-9c45-d0ca32cee422,III,3068308,WBGene00006725,ubl-1,5S5M2D44M1I4M1D4M1D28M1D13M3D28M1I15M1D10M106N...,CCAAGTTTTTAGAACTAGGGTCAGTCAAGCATGGTCTTCGTTAAGA...,NaN,+,-,507
168010,xrn-1-5tera,52e36b5f-9340-4487-b70b-3854ea6781de,III,3068308,WBGene00006725,ubl-1,5S6M1D26M5D4M2D5M1I50M1D3M1D18M1D5M3I32M106N61...,CCAAGTTTTTGAGAACATGGGTCAGTCAAGCATGGTCTAAGACTCC...,25.65,+,-,569
...,...,...,...,...,...,...,...,...,...,...,...,...
532678,xrn-1-5tera-smg-6,52600deb-aa20-4d1b-b0c3-0204ab79e62b,III,3069059,WBGene00006725,ubl-1,8M3D5M3D36M3D7M1D11M1I4M1D4M1I68M4S,CAACAGACACTGCAGATGCCACGACACCCTCGTCGTTGACACCGCC...,NaN,+,+,149
532679,xrn-1-5tera-smg-6,d6db07a6-2cfa-4774-8f00-51ade1bd97ae,III,3069061,WBGene00006725,ubl-1,2S9M1D17M2D6M2I4M1D19M3D1M1I3M1I8M1I39M1D15M1I...,ATACAGACACTCTGCGGAAGATGCCACGTTCCTCGTGTCGTGACAC...,NaN,+,-,159
532680,xrn-1-5tera-smg-6,ac65b7de-76b0-4768-ad6b-9ca768ff8a37,III,3069063,WBGene00006725,ubl-1,12M3D16M1I16M2D25M1D70M26S,AGACACTACTGCAGATGCCACGACACCCACCGTCGTTGACACCGCT...,NaN,+,-,166
532681,xrn-1-5tera-smg-6,13bc5f46-8e83-4ef9-acd2-ff4bb0299365,III,3069072,WBGene00006725,ubl-1,135M97S,TGCGGAAGATGCCACGACACCCTCGTCGTTGACACCGCTACTGCCG...,41.50,+,-,232


In [40]:
# NMD Targets:
gene_violin_list = ['ubl-1', 'rpl-7A', 'odc-1', 'Y73B3A.18','rpl-30', 'rpl-1']
# Not NMD Targets:
# gene_violin_list = ['col-125', 'col-160', 'col-129', 'dod-19', 'vit-3', 'vit-4']

plot_df = reads_df[reads_df['gene_name'].isin(gene_violin_list)].query("lib == 'xrn-1-5tera'").sort_values('gene_name')
# 
# plot_df = reads_df.query("lib == 'xrn-1-5tera'")  #.query("chr_id != 'MtDNA'")

fig = go.Figure()
fig.add_trace(go.Violin(x=plot_df.query("t5 == '+'")['gene_name'],
                        y=plot_df.query("t5 == '+'")['polya_length'],
                        name='t5 +',
                        side='negative',
                        # fillcolor='#fbc6c1',
                        fillcolor='firebrick',
                        spanmode='hard',
                        ))
fig.add_trace(go.Violin(x=plot_df.query("t5 == '-'")['gene_name'],
                        y=plot_df.query("t5 == '-'")['polya_length'],
                        name='t5 -',
                        side='positive',
                        # fillcolor='#c5d8e9',
                        fillcolor='dimgray',
                        spanmode='hard',
                        ))
fig.update_traces(meanline_visible=True,
                  scalemode='width',
                  points='outliers',
                  width=1,
                  line_color='black',
                  box_visible=True)
fig.update_layout(violinmode='overlay',
                  violingap=0,
                  margin={'l': 0, 'b': 40, 't': 10, 'r': 40},
                  yaxis_title=f"Distribution of PolyA Tail Length Calls",
                  template='plotly_white',
                  width=1000, height=450,
                  )
fig.update_yaxes(range=[-5, 201])
fig.write_image(f"./{get_dt(for_file=True)}_splitViolins_exampleGenes.svg")
fig.show()

In [70]:
plot_df['passed_polya'] = ~plot_df['polya_length'].isna()
# fig = sea.countplot(data=plot_df,
#                     x='gene_name',
#                     hue='passed_polya')
# plt.show()
fig = px.histogram(plot_df,
                   x='gene_name',
                   pattern_shape='passed_polya',
                   color_discrete_sequence=['#c5d8e9', '#fbc6c1',],
                   color='t5')
fig.update_layout(barmode='relative')
fig.update_layout(margin={'l': 0, 'b': 40, 't': 10, 'r': 40},
                  template='plotly_white')
fig.show()